<a href="https://colab.research.google.com/github/vukhanhlinh/atom-assignments/blob/main/assignment_3/home_assignment_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HOME ASSIGNMENT #3: SLACK API - TO GSHEET
**Mục đích của bài Assignment**
- Lấy thông tin các Users từ Slack của DataCracy (BTC, Mentors và Learners)
- `**[Optional 1]**` Đưa danh sách Users lên Google Spreadsheet, để theo dõi 
- `**[Optional 2]**` Lấy thông tin Assignment Submission và số Reviews trên `#atom-assignmentnt2` và cập nhật lên Spreadsheet, để theo dõi các học viên đã nộp bài và được review

**Các kiến thức sẽ áp dụng**
- Ôn lại và luyện tập thêm về concept API (cụ thể sử dụng API Slack)
- Trích xuất thông tin từ JSON
- Dùng module gspread để đưa thông tin lên Google Spreadsheet

## 0. Load Modules

In [36]:
import requests #-> Để gọi API
import re #-> Để xử lý data dạng string
from datetime import datetime as dt #-> Để xử lý data dạng datetime
import gspread #-> Để update data lên Google Spreadsheet
from gspread_dataframe import set_with_dataframe #-> Để update data lên Google Spreadsheet
import pandas as pd #-> Để update data dạng bản
import json 
from oauth2client.service_account import ServiceAccountCredentials #-> Để nhập Google Spreadsheet Credentials
import os

## 1. Slack API: User List
* Bạn có thể đọc lại về concept API [HERE](https://anhdang.gitbook.io/datacracy/atom/3-data-tools-2/3.2-spotify-api-and-postman)
* Assignment này sẽ dùng Slack API để lấy thông tin về Learners và theo dõi các bài tập đã nộp và được review (sau đó cập nhật lên Google Spreadsheet)
* ===> **NOTICE**: Slack API authorize bằng Bearer Token `xoxb-...-...-...` (Sẽ được cung cấp riêng)
* Update file `env_variable.json` như trong [Assignment#2](../assignment_2/home_assignment_2.ipynb)
* ==> Nếu bạn dùng Google Colab, upload file vào Colab ([Hướng dẫn](https://colab.research.google.com/notebooks/io.ipynb))

In [37]:
!ls #Liệt kê các file

datacracy-assignment3.json  env_variable.json  sample_data


In [38]:
with open('env_variable.json', 'r') as j:
    json_data = json.load(j)
    # print(json_data)

In [39]:
## Load SLACK_BEARER_TOKEN
os.environ['SLACK_BEARER_TOKEN'] = json_data['SLACK_BEARER_TOKEN'] 

In [40]:
## Gọi API từ Endpoints (Input - Token được đưa vào Headers)
## Challenge: Thử gọi API này bằng Postman
endpoint = "https://slack.com/api/users.list"
headers = {"Authorization": "Bearer {}".format(os.environ['SLACK_BEARER_TOKEN'])}
response_json = requests.post(endpoint, headers=headers).json() 
user_dat = response_json['members']

### TODO #1
Hoàn tất đoạn code sau

In [41]:
user_dat #Liệt kê thông tin 

[{'color': '757575',
  'deleted': False,
  'id': 'USLACKBOT',
  'is_admin': False,
  'is_app_user': False,
  'is_bot': False,
  'is_email_confirmed': False,
  'is_owner': False,
  'is_primary_owner': False,
  'is_restricted': False,
  'is_ultra_restricted': False,
  'name': 'slackbot',
  'profile': {'always_active': True,
   'avatar_hash': 'sv41d8cd98f0',
   'display_name': 'Slackbot',
   'display_name_normalized': 'Slackbot',
   'fields': None,
   'first_name': 'slackbot',
   'image_192': 'https://a.slack-edge.com/80588/marketing/img/avatars/slackbot/avatar-slackbot.png',
   'image_24': 'https://a.slack-edge.com/80588/img/slackbot_24.png',
   'image_32': 'https://a.slack-edge.com/80588/img/slackbot_32.png',
   'image_48': 'https://a.slack-edge.com/80588/img/slackbot_48.png',
   'image_512': 'https://a.slack-edge.com/80588/img/slackbot_512.png',
   'image_72': 'https://a.slack-edge.com/80588/img/slackbot_72.png',
   'last_name': '',
   'phone': '',
   'real_name': 'Slackbot',
   'real_

In [42]:
## Loop qua JSON file và extract các thông tin quan trọng (id, name, display_name, real_name_normalized, title, phone, is_bot)
## Hint: Bạn có thể dùng Postman hoặc in user_dat JSON để xem cấu trúc (schema), dùng Ctrl+F để tìm các keys
#        (id, name, display_name, real_name_normalized, title, phone, is_bot)
user_dict = {'user_id':[], 'name':[], 'display_name':[],'real_name':[],'title':[],'phone':[],'is_bot':[]}
for i in range(len(user_dat)):
  user_dict['user_id'].append(user_dat[i]['id'])
  user_dict['name'].append(user_dat[i]['name'])
  user_dict['display_name'].append(user_dat[i]['profile']['display_name'])
  user_dict['real_name'].append(user_dat[i]['profile']['real_name_normalized'])
  user_dict['title'].append(user_dat[i]['profile']['title'])
  user_dict['phone'].append(user_dat[i]['profile']['phone'])
  user_dict['is_bot'].append(user_dat[i]['is_bot'])

In [43]:
user_df = pd.DataFrame(user_dict) ## Dùng pandas để convert dictionaries thành bảng
user_df.head(5) ## Chỉ in 5 dòng đầu (chủ yếu để xem cấu trúc)

,user_id,name,display_name,real_name,title,phone,is_bot
0,USLACKBOT,slackbot,Slackbot,Slackbot,,,False
1,U01AT4T75JB,loclexuan26392,Loc Le Xuan,Loc Le Xuan,,,False
2,U01AVDY7JET,locle.ds,Loc Le Xuan,Loc Le Xuan,,,False
3,U01BE2PR6LU,maianhdang.ftu,MAD,Dặng Huỳnh Mai Anh,Technical Contents,,False
4,U01C48T7S1J,huyenhoang.design,Thanh Huyen Hoang,Thanh Huyen Hoang,,,False


In [44]:
user_df[user_df.display_name == 'MAD'] ## Lọc thông tin của MAD, trên DataFrame (bạn có thể Google thêm)

,user_id,name,display_name,real_name,title,phone,is_bot
3,U01BE2PR6LU,maianhdang.ftu,MAD,Dặng Huỳnh Mai Anh,Technical Contents,,False


-------------- HẾT PHẦN BẮT BUỘC ---------------------

## Option 1: Update data => Google SpreadSheet

### TODO#2
Tạo service account (output là file json), file này để cho phép ta access vào Google Spreadsheet:

1. Làm theo hướng dẫn: [Google Create a Service Account](https://support.google.com/a/answer/7378726?hl=en)
![google_service_account](https://github.com/vukhanhlinh/atom-assignments/blob/main/img/google_service_account.png?raw=1)
2. Lưu file JSON (chứa credential về máy)
![gservice_acc_json](https://github.com/vukhanhlinh/atom-assignments/blob/main/img/gservice_acc_json.png?raw=1)
3. Nhớ Enable [Google Drive API](https://console.cloud.google.com/marketplace/product/google/drive.googleapis.com?q=search&referrer=search&project=quickstart-313303) (Nếu bạn chạy code báo lỗi chưa enable API thì vào link trong phần lỗi để Enable, sau khi kích hoạt có thể cần vài phút để chạy được)
![enable_api](https://github.com/vukhanhlinh/atom-assignments/blob/main/img/enable_api.png?raw=1)
* ==> Upload file Gsheet Credential JSON nếu bạn dùng Colab 
* ==> Nếu bạn để key trong repo git, **NHỚ** để file json vào `.gitignore` để không bị leaked key)


In [45]:
!ls

datacracy-assignment3.json  env_variable.json  sample_data


In [46]:
## Authorize bằng JSON
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('datacracy-assignment3.json', scope)
gc = gspread.authorize(credentials)
print("DONE!")

DONE!


**Tạo Spreadsheet**

1. Tạo Spreadsheet trên google
2. Invite account trong `client_email` (file JSON Gsheet Credential bên trên) vào Spreadsheet (quyền Editor)
![enable_api](https://github.com/vukhanhlinh/atom-assignments/blob/main/img/enable_api.png?raw=1)
3. Lấy `SPREADSHEET_KEY` (nằm trong chính URL của Spreadhstee): `https://docs.google.com/spreadsheets/d/<SPREADSHEET_KEY>/edit#gid=0`

![add_gsheet](https://github.com/vukhanhlinh/atom-assignments/blob/main/img/add_gsheet.png?raw=1)

In [47]:
# ACCES GOOGLE SHEET
sheet_index_no = 0
spreadsheet_key = '1hNprbtIOKbtyQ7rsG7Ewy566QxlwJfnVxW9QBt38cWQ' # input SPREADSHEET_KEY HERE
sh = gc.open_by_key(spreadsheet_key)
worksheet = sh.get_worksheet(sheet_index_no) #-> 0 - first sheet, 1 - second sheet etc. 

# APPEND DATA TO SHEET
set_with_dataframe(worksheet, user_df) #-> Upload user_df vào Sheet đầu tiên trong Spreadsheet

# DONE: Bây giờ bạn có thể mở spreadsheet và kiểm tra nội dung đã update chứ

![slack_user_gsheet](https://github.com/vukhanhlinh/atom-assignments/blob/main/img/slack_user_gsheet.png?raw=1)

-------------- HẾT PHẦN OPTION 1 ---------------------

## Option 2: Ai đã nộp bài?


### Slack API: Channel List

In [87]:
## Gọi SLACK API để list tất cả các channel
endpoint = "https://slack.com/api/conversations.list"
headers = {"Authorization": "Bearer {}".format(os.environ['SLACK_BEARER_TOKEN'])}
response = requests.post(endpoint, headers=headers).json() 
channel_ls = response['channels']

In [88]:
channel_ls[0] ## Thử extract record đầu tiên để xem schema  => name: general, id: C01B4PVGLVB

{'created': 1600856703,
 'creator': 'U01BE2PR6LU',
 'id': 'C01B4PVGLVB',
 'is_archived': False,
 'is_channel': True,
 'is_ext_shared': False,
 'is_general': True,
 'is_group': False,
 'is_im': False,
 'is_member': False,
 'is_mpim': False,
 'is_org_shared': False,
 'is_pending_ext_shared': False,
 'is_private': False,
 'is_shared': False,
 'name': 'general',
 'name_normalized': 'general',
 'num_members': 66,
 'parent_conversation': None,
 'pending_connected_team_ids': [],
 'pending_shared': [],
 'previous_names': [],
 'purpose': {'creator': 'U01BE2PR6LU',
  'last_set': 1600856703,
  'value': 'This is the one channel that will always include everyone. It’s a great spot for announcements and team-wide conversations.'},
 'shared_team_ids': ['T01B7SGGMLJ'],
 'topic': {'creator': '', 'last_set': 0, 'value': ''},
 'unlinked': 0}

### TODO#3 
* Tìm id của channel #atom-assignment2

In [89]:
for i in channel_ls:
    print(i['name'])


general
contents
branding-design
atom-assignment1
atom-week1
atom-general
topics-data-analytics
topics-python
topics-materials
atom-assignment2
atom-week2
atom-week3
atom-assignment3
atom-assignment4
atom-week4
atom-assignment5
atom-week5
atom-week6
atom-assignment6


In [90]:
for i in channel_ls:
    if (i['name']) == 'atom-assignment2':
        print('atom-assignment2 id is: ', i['id'])

atom-assignment2 id is:  C021FSDN7LJ


# **Slack API: List messages trong 1 channel**

In [91]:
## Gọi SLACK API
endpoint = "https://slack.com/api/conversations.history"
data = {"channel": "C01U6P7LZ8F"} ## This is ID of assignment#1 channel
headers = {"Authorization": "Bearer {}".format(os.environ['SLACK_BEARER_TOKEN'])}

In [92]:
response_json = requests.post(endpoint, data=data, headers=headers).json()
msg_ls = response_json['messages']

In [93]:
msg_ls[21]

{'blocks': [{'block_id': 'aluFq',
   'elements': [{'elements': [{'text': 'Cám ơn Quân nhiều nhé. Hay quá Quân ơi',
       'type': 'text'}],
     'type': 'rich_text_section'}],
   'type': 'rich_text'}],
 'client_msg_id': '21094a1f-9682-4838-96c7-17735444db09',
 'reactions': [{'count': 1, 'name': 'heart', 'users': ['U01UMJ36QD9']}],
 'root': {'attachments': [{'fallback': 'HackerRank: Between Two Sets | HackerRank',
    'from_url': 'https://www.hackerrank.com/challenges/between-two-sets/problem',
    'id': 1,
    'original_url': 'https://www.hackerrank.com/challenges/between-two-sets/problem',
    'service_icon': 'https://hrcdn.net/community-frontend/assets/favicon-ddc852f75a.png',
    'service_name': 'HackerRank',
    'text': 'Find the number of integers that satisfies certain criteria relative to two sets.',
    'thumb_height': 640,
    'thumb_url': 'https://hrcdn.net/og/default.jpg',
    'thumb_width': 1200,
    'title': 'Between Two Sets | HackerRank',
    'title_link': 'https://www.h

In [133]:
not_learners_id = ['U01BE2PR6LU']

In [134]:
## Summarize all submitted assignments + reviews cnt
not_learners_id = ['U01BE2PR6LU'] # -> Remove MA from the user_id
github, reply_count, reply_users_count, reply_users, latest_reply = '','','','',''
for i in range(20):
  ts = dt.fromtimestamp(float(msg_ls[i]['ts'])) # -> Convert timestamp Epoch thành dàng dễ đọc
  user = msg_ls[i]['user'] # -> Lấy thông tin người post messages
  if msg_ls[i]['user'] not in not_learners_id:
    if 'attachments' in msg_ls[i].keys():
      #print(msg_ls[i].keys())
      text = msg_ls[i]['text']
      github_link = re.findall('(?:https?://)?(?:www[.])?github[.]com/[\w-]+/?', text) #-> Submission là các message có link github
      #print(msg_ls[i])
      if len(github_link) > 0: github = github_link[0]
      if 'reply_count' in msg_ls[i].keys(): reply_count = msg_ls[i]['reply_count'] #-> Extract số review
      if 'reply_users_count' in msg_ls[i].keys(): reply_users_count = msg_ls[i]['reply_users_count']
      if 'reply_users' in msg_ls[i].keys(): reply_users = msg_ls[i]['reply_users']
      if 'latest_reply' in msg_ls[i].keys(): latest_reply = dt.fromtimestamp(float(msg_ls[i]['latest_reply']))
      print(ts, user, reply_users_count, reply_users, latest_reply, github)

2021-05-10 04:51:46.004700 U01UJ9LG5U5 2 ['U01USGKQ771', 'U01V00KHHHP'] 2021-05-15 09:14:31.003600 https://github.com/danhpcv/
2021-05-10 04:35:39.003800 U01UMC08KL2 2 ['U01UTGS5ZNY', 'U01UMC08KL2'] 2021-05-18 08:09:59.000800 https://github.com/auslynnguyen/


### TODO#4
* Tạo thành 1 bảng chứa các thông tin trên và update lên Spreadsheet (Sheet: Assignment#2 Submission)

In [135]:
## Gọi SLACK API
endpoint = "https://slack.com/api/conversations.history"
data = {"channel": "C01U6P7LZ8F"} ## This is ID of assignment#1 channel
headers = {"Authorization": "Bearer {}".format(os.environ['SLACK_BEARER_TOKEN'])}

In [136]:
##Lấy message
response_json = requests.post(endpoint, data=data, headers=headers).json()
msg_as2 = response_json['messages']

In [137]:
not_learners_id = ['U01BE2PR6LU'] #LoạiLoại MA ra khỏi user_id

In [150]:
msg_as2[34].keys()


dict_keys(['client_msg_id', 'type', 'text', 'user', 'ts', 'team', 'edited', 'attachments', 'blocks'])

In [139]:
msg_as2

[{'inviter': 'U01BE2PR6LU',
  'subtype': 'channel_join',
  'text': '<@U0226MQ73MJ> has joined the channel',
  'ts': '1622694670.000600',
  'type': 'message',
  'user': 'U0226MQ73MJ'},
 {'inviter': 'U01BE2PR6LU',
  'subtype': 'channel_join',
  'text': '<@U0226MQ73MJ> has joined the channel',
  'ts': '1622616102.000300',
  'type': 'message',
  'user': 'U0226MQ73MJ'},
 {'inviter': 'U01BE2PR6LU',
  'subtype': 'channel_join',
  'text': '<@U0226MQ73MJ> has joined the channel',
  'ts': '1622216202.000300',
  'type': 'message',
  'user': 'U0226MQ73MJ'},
 {'inviter': 'U01BE2PR6LU',
  'subtype': 'channel_join',
  'text': '<@U0226MQ73MJ> has joined the channel',
  'ts': '1621856793.004500',
  'type': 'message',
  'user': 'U0226MQ73MJ'},
 {'inviter': 'U01BE2PR6LU',
  'subtype': 'channel_join',
  'text': '<@U0226MQ73MJ> has joined the channel',
  'ts': '1621756508.004100',
  'type': 'message',
  'user': 'U0226MQ73MJ'},
 {'inviter': 'U01BE2PR6LU',
  'subtype': 'channel_join',
  'text': '<@U0226MQ73M

In [140]:
dict_as2 = {'ts':[], 'user':[], 'reply_count':[],'reply_users':[],'latest_reply':[],'github':[]} #Liệt kê các keys
not_learners_id = ['U01BE2PR6LU'] # -> Remove MA from the user_id
# user_dict_as2

In [151]:
github, reply_count, reply_users_count, reply_users, latest_reply = '','','','',''
for i in range(len(msg_as2)):
  ts = dt.fromtimestamp(float(msg_as2[i]['ts'])) # -> Convert timestamp Epoch thành dàng dễ đọc
  user = msg_as2[i]['user'] # -> Lấy thông tin người post messages
  if msg_as2[i]['user'] not in not_learners_id:
    if 'attachments' in msg_as2[i].keys():
      # print(msg_as2[i].keys())
      text = msg_as2[i]['text']
      github_link = re.findall('(?:https?://)?(?:www[.])?github[.]com/[\w-]+/?', text) #-> Submission là các message có link github
      #print(msg_as2[i])
      if len(github_link) > 0: github = github_link[0]
      if 'user' in msg_as2[i].keys():  user_dict_as2['user'].append(msg_as2[i]['user'])
      else:  user_dict_as2['user'].append('No data')
      if 'reply_count' in msg_as2[i].keys():  user_dict_as2['reply_count'].append(msg_as2[i]['reply_count'])#-> Extract số review 
      else: user_dict_as2['reply_count'].append('No data')
      if 'reply_users' in msg_as2[i].keys(): user_dict_as2['reply_users'].append(msg_as2[i]['reply_users'])
      else: user_dict_as2['reply_users'].append('No data')
      if 'latest_reply' in msg_as2[i].keys(): user_dict_as2['latest_reply'].append(msg_as2[i]['latest_reply'])
      else: user_dict_as2['latest_reply'].append('No data')
      print(ts, user, reply_users_count, reply_users, latest_reply, github)


2021-05-10 04:51:46.004700 U01UJ9LG5U5    https://github.com/danhpcv/
2021-05-10 04:35:39.003800 U01UMC08KL2    https://github.com/auslynnguyen/
2021-05-08 06:31:59.005200 U01UMJ36QD9    https://github.com/auslynnguyen/
2021-05-04 13:55:46.001800 U01UTGVPE7N    https://github.com/hoaintp/
2021-05-03 17:41:18.002400 U01U6JQB695    https://github.com/Tenderriver/
2021-05-03 08:51:19.001200 U01V00KHHHP    https://github.com/saturn1101/
2021-04-26 06:53:56.000800 U01UMBZHU2W    https://github.com/saturn1101/
2021-04-24 02:01:27.043200 U01VB63LRNC    https://github.com/AnhThuNguyenHuynh/
2021-04-24 01:08:57.041400 U01U6JM6LEB    https://github.com/vietqh/
2021-04-23 22:16:41.038200 U01V00P4S2V    https://github.com/vuthanhdatt/
2021-04-23 18:56:58.034700 U01UTGRP0HJ    https://github.com/TranPham-96/
2021-04-23 16:45:14.029300 U01UMJ451V1    https://github.com/annaho124/
2021-04-23 16:41:53.026200 U01UJ9M5TU5    https://github.com/minhnguyentk95/
2021-04-23 15:48:06.023900 U01UJ9NKX8D    ht

In [152]:
len(user_dict_as2['user'])

154

In [156]:
as2_df = pd.DataFrame(user_dict_as2) ## Dùng pandas để convert dictionaries thành bảng
# as2.head(5) ## Chỉ in 5 dòng đầu (chủ yếu để xem cấu trúc)

ValueError: ignored

In [157]:
# ACCES GOOGLE SHEET
sheet_index_no = 0
spreadsheet_key = '164_OwrRR10m5N5-avjMn1fBZhgBq6qNpFA6ay0uZuY8' # input SPREADSHEET_KEY HERE
sh = gc.open_by_key(spreadsheet_key)
worksheet = sh.get_worksheet(sheet_index_no) #-> 0 - first sheet, 1 - second sheet etc. 

# APPEND DATA TO SHEET
set_with_dataframe(worksheet, user_df) #-> Upload user_df vào Sheet đầu tiên trong Spreadsheet

# DONE: Bây giờ bạn có thể mở spreadsheet và kiểm tra nội dung đã update chứ

APIError: ignored

-------------- HẾT PHẦN OPTION 2 ---------------------